In [25]:
from langgraph.graph import StateGraph, START, END
from openai import OpenAI
from dotenv import load_dotenv
from typing import TypedDict
import os

In [17]:
load_dotenv()

False

In [20]:
# Get the key from env variable
groq_key = os.getenv("GROQ_API_KEY")

In [21]:
print("Loaded key:", groq_key is not None)  # Debug: should print Tru

Loaded key: False


In [19]:
model = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [22]:
# create a state 

class LLMState(TypedDict):

    question:str
    answer:str

In [27]:
# create a function to ask the llm 
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from the state 
    question = state["question"]

    # form a prompt
    prompt = f"Answer the following question: {question}"

    # ask that question to the llm
    answer = model.invoke(prompt).content

    # update the state with the answer 
    state["answer"] = answer

    return state

In [ ]:
# create our graph 

graph = StateGraph(LLMState)

# create our nodes 
graph.add_node("llm_qa", llm_qa)

#create our edges 
graph.add_edge(START, "llm_qa")
graph.add_edge("llm_qa", END)

#compile
workflow = graph.compile()


In [30]:
#run the workflow 

initial_state = {"question": "What is the capital of France?"}

final_state = workflow.invoke(initial_state)

print(final_state["answer"])

NameError: name 'model' is not defined